# Deloitte Quantum Challenge 2023

# =============================

# 0. References and credits

In [ ]:
'''
Team: Quantux
Members: ...

V00 20230317 - JW - Phase 1: Establishment
V01 20230226 - JW - Phase 1: Add various local & realtime VQE client
V02 20230301 - JW - Phase 1: Add decorators and modularise pipeline (Fix decorators later)
V03 20230308 - JW - Phase 1: Fix optimizers, add runtime providers, add fake providers
'''

In [ ]:
'''
References:
# Python functions
https://dev.to/kcdchennai/python-decorator-to-measure-execution-time-54hk

# Machine Learning modules
https://towardsdatascience.com/state-of-the-art-machine-learning-hyperparameter-optimization-with-optuna-a315d8564de1

# Qiskit with latest post-migration modules and functions
https://github.com/PacktPublishing/Quantum-Chemistry-and-Computing-for-the-Curious/blob/main/Chapter_05_Variational_Quantum_Eigensolver_.VQE._algorithm_V2.ipynb

https://qiskit.org/documentation/nature/_modules/qiskit_nature/second_q/
https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html
https://qiskit.org/documentation/nature/tutorials/05_problem_transformers.html

# Qiskit realtime IBQM providers
https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.AccountProvider.html
https://qiskit.org/documentation/partners/qiskit_ibm_provider/tutorials/Migration_Guide_from_qiskit-ibmq-provider.html
https://quantumcomputing.stackexchange.com/questions/17375/is-there-any-way-to-obtain-the-number-of-qubits-of-a-given-backend-in-qiskit
https://qiskit.org/documentation/apidoc/providers_fake_provider.html
https://qiskit.org/documentation/tutorials/algorithms/02_vqe_advanced_options.html
https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials/Error-Suppression-and-Error-Mitigation.html
https://qiskit.org/documentation/tutorials/algorithms/03_vqe_simulation_with_noise.html

https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-mitigation.html
https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.DynamicalDecoupling.html

Options - resilience level: No error mitigation
Options - optimization level: 

# Distance: dist on metal ion + gas molecule

'''

# =============================

# 1. Import libraries and qiskit functions

In [1]:
# 0. Data science & ML libraries
# =============================
import pandas as pd
import numpy as np
import warnings, os, time
import matplotlib.pyplot as plt
from tqdm import tqdm

warnings.filterwarnings('ignore')

from functools import wraps
from qiskit.utils import algorithm_globals

from functools import partial
from scipy.optimize import minimize

_SEED = 5
np.random.seed(_SEED)
algorithm_globals.random_seed = _SEED

In [2]:
# All are post-migration imports, ie. qiskit_nature == 0.5.0
import qiskit_ibm_provider, qiskit, qiskit_nature
v_provider = qiskit_ibm_provider.__version__
v_qiskit = qiskit.__version__
v_qiskit_nature = qiskit_nature.__version__

if (v_provider >= '0.1') and (v_qiskit >= '0.22') and (v_qiskit_nature >= '0.5'):
    
    # 1. Load qiskit account & providers
    # =============================
    from qiskit_ibm_provider import IBMProvider
    provider = IBMProvider(instance="deloitte-event23/level-1-access/quantux")
    deloitte_provider = provider
    # deloitte_simulators = []
    from qiskit.providers.fake_provider import FakeProvider
    # from qiskit.providers.fake_provider import FakeVigo
    # from qiskit.test.mock import FakeProvider
    fake_provider = FakeProvider()
    
    # provider.backends() # List all backend providers
    qasm_simulator = provider.get_backend('ibmq_qasm_simulator')
    statevector_simulator = provider.get_backend('simulator_statevector')
    
    from qiskit.providers.aer import StatevectorSimulator
    from qiskit import Aer
    from qiskit_aer.noise import NoiseModel
    
    from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session
    

    # 2. Molecule & backend definition
    # =============================
    from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
    from qiskit_nature.units import DistanceUnit
    from qiskit.utils import QuantumInstance
    from qiskit_nature.second_q.drivers import PySCFDriver

    # VQE Client seem can't be used, use VQE instead
    # from qiskit_nature.runtime import VQEClient
    
    
    # 3. Estimators & Sampler Primitives
    # =============================
    from qiskit.primitives import Estimator, Sampler # Noiseless
    from qiskit_aer.primitives import Estimator as AerEstimator # Noisy
    
    
    # 4. Ansatz, initial circuit libraries, initial points (Ansatz parameters)
    # =============================
    from qiskit_nature.second_q.circuit.library import UCC, UCCSD, PUCCD, SUCCD, UVCCSD # Ansatz
    from qiskit_nature.second_q.circuit.library import HartreeFock # Initial State
    from qiskit.circuit.library import TwoLocal, PauliTwoDesign, EfficientSU2, RealAmplitudes, ExcitationPreserving # Initial Circuit
    from qiskit.quantum_info import Pauli
    #from qiskit.circuit.library.initial_states import hartree_fock, fermionic_gaussian_state, vscf
    #from qiskit.circuit.library.initial_points import mp2_initial_point, hf_initial_point, mp2_initial_point, vscf_initial_point
    
    
    # 5. Qubit Mappers & transformers
    # =============================
    from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, BravyiKitaevMapper, ParityMapper
    from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
    
    
    # 6. Algorithms & optimizers
    # =============================
    from qiskit_nature.second_q.algorithms import NumPyMinimumEigensolverFactory, VQEUCCFactory, VQEUVCCFactory
    from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
    from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
    from qiskit.algorithms.optimizers import COBYLA, SLSQP, SPSA, QNSPSA

    from qiskit.algorithms import HamiltonianPhaseEstimation, PhaseEstimation
    from qiskit.opflow import StateFn, PauliExpectation, CircuitSampler, PauliTrotterEvolution

else:
    print('Qiskit version not mostly updated - v2. Need updates before continue.')
    pass

# =============================

# 2. Define construct problem modules

# 2-0. Decorators

In [3]:
# Timeit decorator
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} took {total_time:.4f} seconds to run\n')
        return result
    return timeit_wrapper


In [4]:
# Try except decorator
def get_err_msg_silent(value):
    def decorate(f):
        def applicator(_silent = False, *args, **kwargs):
            try:
                if not _silent:
                    print('{}: Loading...'.format(f.__name__))
                
                res = f(*args,**kwargs)
                
                if not _silent:
                    print('Success in loading {}'.format(f.__name__))
                return res
                
            except:
                if not _silent:
                    print('Fail in loading {}'.format(f.__name__))
                return value
        return applicator
    return decorate

In [5]:
# Try except decorator
def get_err_msg(value):
    def decorate(f):
        def applicator(*args, **kwargs):
            try:
                print('{}: Loading...'.format(f.__name__))
                res = f(*args,**kwargs)
                return res
            except:
                return value
        return applicator
    return decorate

# 2-1. Define modular functions

In [6]:
# Get a random FakeProvider from IBM runtime
def _0_get_min_runtime_provider(provider, min_qubits):
    
    # fake_provider = FakeProvider()
    # deloitte_provider = IBMProvider(instance="deloitte-event23/level-1-access/quantux")
    try:
        provider_list = [[b.configuration().n_qubits, b.name()] for b in provider.backends() if b.configuration().n_qubits >= min_qubits] # Usable in Aer
    except:
        provider_list = [[b.configuration().n_qubits, b.name] for b in provider.backends() if b.configuration().n_qubits >= min_qubits] # Usable in FakeProvider

    from collections import defaultdict
    provider_dict = defaultdict(list)
    for k, v in provider_list:
        provider_dict[k].append(v)

    provider_dict = dict(provider_dict)

    min_qubit = min(i for i in list((provider_dict.keys())) if i > min_qubits)

    provider_list_chosen = provider_dict[min_qubit]
    idx = np.random.randint(len(provider_list_chosen)) # Random choose provider with equivalent qubit computing size
    provider_out=provider_list_chosen[idx]

    print('Noisy Aer provider with min_qubit chosen: {}, {}'.format(min_qubit, provider_out))

    return provider_out

#_0_get_min_runtime_provider(provider, min_qubits = 5) # Deloitte provider / IBM main provider
#_0_get_min_runtime_provider(fake_provider, min_qubits = 5)

In [7]:
# 1. Get problem driver
#@get_err_msg('')
#@timeit
def _1_get_problem_driver(display_dict, input_value, basis = 'sto3g', input_type = 'molecule'):
    
    # =============================
    # 1. Define Molecule dictionary and return problem from PySCFDriver
    # =============================
    
    # =============================
    if input_type == 'molecule':
        moleculeinfo = MoleculeInfo(symbols = input_value['symbols'], coords = input_value['coords'], masses = input_value['masses'], 
                                    charge = input_value['charge'], multiplicity = input_value['multiplicity'])
    
    elif input_type == 'moleculeinfo':
        moleculeinfo = input_value
    
    driver = PySCFDriver.from_molecule(moleculeinfo, basis=basis)
    problem = driver.run()
    
    
    if display_dict is not None:
        display_dict['molecule'] = '{}'.format(input_value['symbols'])
        display_dict['charge'] = '{}'.format(input_value['charge'])
        display_dict['multiplicity'] = '{}'.format(input_value['multiplicity'])
        
        display_dict['reference_energy'] = '{}'.format(problem.reference_energy)
        display_dict['num_spin_orbitals'] = '{}'.format(problem.num_spin_orbitals)
        display_dict['num_spatial_orbitals'] = '{}'.format(problem.num_spatial_orbitals)
        display_dict['num_particles'] = '{}'.format(problem.num_particles)
        display_dict['nuclear_repulsion_energy'] = '{}'.format(problem.nuclear_repulsion_energy)
        display_dict['num_alpha'] = '{}'.format(problem.num_alpha)
        display_dict['num_beta'] = '{}'.format(problem.num_beta)
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return problem, display_dict_new

In [8]:
# 1. Get transform problem
#@get_err_msg('')
#@timeit
def _2_get_problem_transform(display_dict, problem, input_value, reduced):
    
    # =============================
    # 2. Transform the problem to reduce simulation space
    # =============================
    
    # =============================
    # If enter gas_molecules info, can transform; else if enter molecularinfo info, can not transform (should have previously transformed)
    
    # Problem reduction
    if reduced == 'FreezeCore':
        try:
            fc_transformer = FreezeCoreTransformer(freeze_core = input_value['fc_transformer']['fc_freeze_core'], 
                                                   remove_orbitals = input_value['fc_transformer']['fc_remove_orbitals'])
            problem = fc_transformer.transform(problem)
            display_chosen = input_value['fc_transformer']['fc_remove_orbitals']
        except:
            #print('FreezeCore Transformer did not succeed.')
            if display_dict is not None:
                display_dict['FreezeCoreTransformer'] = 'error'
            pass
        
        # How to determine which orbitals to be removed
        # https://quantumcomputing.stackexchange.com/questions/17852/use-one-body-integrals-to-know-which-orbitals-to-freeze-in-electronicstructurepr
        # https://www.youtube.com/watch?v=3B04KB0pDwE&t=667s
        
    elif reduced == 'ActiveSpace':
        #max_num_spatial_orbitals = problem.num_spatial_orbitals
        #max_num_electrons = problem.num_electrons
        #max_active_orbitals = itertools.combinations(max_num_spatial_orbitals, 2)
        # Use optuna to setup objective
        
        try:
            as_transformer = ActiveSpaceTransformer(num_electrons = input_value['as_transformer']['as_num_electrons'], 
                                                    num_spatial_orbitals = input_value['as_transformer']['as_num_spatial_orbitals'], 
                                                    active_orbitals = input_value['as_transformer']['as_active_orbitals'])
            problem = as_transformer.transform(problem)
            display_chosen = input_value['as_transformer']['as_active_orbitals']
        except:
            #print('ActiveSpace Transformer did not succeed.')
            if display_dict is not None:
                display_dict['ActiveSpaceTransformer'] = 'error'
            pass
    else:
        # print('Expect lengthy simulation if can not succeed in reducing orbitals using FreezeCoreTransformer or ActiveSpaceTransformer.')
        display_chosen = ''
        pass
    
    
    if display_dict is not None:
        display_dict['reduction_method'] = '{}'.format(reduced)
        display_dict['orbitals_removed'] = '{}'.format(display_chosen)
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return problem, display_dict_new

In [9]:
# 3. Get qubit operator
#@get_err_msg('')
#@timeit
def _3_get_qubit_operator(display_dict, problem, hyperparam, mapper_type):
    
    # =============================
    # 3. Define qubit mapping and convert to qubit operator
    # =============================
    
    # =============================
    # Qubit mapping
    if mapper_type == 'ParityMapper':
        mapper = ParityMapper()
    elif mapper_type == 'JordanWignerMapper':
        mapper = JordanWignerMapper()
    elif mapper_type == 'BravyiKitaevMapper':
        mapper = BravyiKitaevMapper()
    
    fermionic_hamiltonian = problem.hamiltonian
    second_q_op = fermionic_hamiltonian.second_q_op()
    
    num_particles = problem.num_particles
        
    qubit_converter = QubitConverter(mapper, 
                                     two_qubit_reduction = hyperparam['two_qubit_reduction'], 
                                     z2symmetry_reduction = hyperparam['z2symmetry_reduction'])
    qubit_op = qubit_converter.convert(second_q_op, num_particles = num_particles, sector_locator = problem.symmetry_sector_locator)
    
    
    if display_dict is not None:
        display_dict['second_q_op'] = '{}'.format("\n".join(str(second_q_op).splitlines()[:10] + ["..."]))
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
        
    return qubit_op, qubit_converter, display_dict_new

In [10]:
# 4. Get quantum problem solver
#@get_err_msg('')
#@timeit
def _4_get_ansatz(display_dict, problem, hyperparam, qubit_op, qubit_converter, seed, randomize = False):
    
    # =============================
    # 4. Define various solver types with initalizing the ansatz or initial circuit
    # =============================
    
    # =============================
    num_spatial_orbitals = problem.num_spatial_orbitals
    num_particles = problem.num_particles
    num_qubits = qubit_op.num_qubits
    
    # i. List of ansatz
    # [UCCSD(), UCC(), PUCCD(), SUCCD(), UVCCSD()] # Others like PUCCD needs alpha == beta, ie. lots of restrictions thus all except UCCSD are left unused
    ansatz_uccsd = UCCSD()
    ansatz_uccsd.num_spatial_orbitals = num_spatial_orbitals
    ansatz_uccsd.num_particles = num_particles
    ansatz_uccsd.qubit_converter = qubit_converter
    
    ansatz_hf = HartreeFock(num_spatial_orbitals, num_particles, qubit_converter)
    
    ansatz_lst = [ansatz_uccsd, ansatz_hf]
    ansatz_name = ['UCCSD', 'HartreeFock']
    
    # =============================
    # ii. For QNSPSA, must match num_qubits to the circuit observable, currently H2 is 4, use this constraint to subordinate for num_qubits' design from initial_circuits
    
    circuit_tl = TwoLocal(num_qubits, rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz', entanglement='full', reps=hyperparam['reps'], parameter_prefix = 'y')
    circuit_su2 = EfficientSU2(num_qubits, reps=hyperparam['reps'], entanglement="full")
    circuit_p2d = PauliTwoDesign(num_qubits, reps=hyperparam['reps'], seed=seed)
    circuit_ra = RealAmplitudes(num_qubits, reps=hyperparam['reps'])
    circuit_ep = ExcitationPreserving(num_qubits, entanglement = 'full', reps=hyperparam['reps'], mode = 'iswap')
    
    initial_circuit_lst = [circuit_tl, circuit_su2, circuit_p2d, circuit_ra, circuit_ep]
    initial_circuit_name = ['TwoLocal', 'EfficientSU2', 'PauliTwoDesign', 'RealAmplitudes', 'ExcitationPreserving']
    
    if randomize:
        idx_ansatz = np.random.randint(len(ansatz_lst))
        ansatz_chosen = ansatz_lst[idx_ansatz]
        
        idx_initial_circuit = np.random.randint(len(initial_circuit_lst))
        initial_circuit_chosen = initial_circuit_lst[idx_initial_circuit]
    else:
        an = hyperparam['ansatz']
        an_val = np.where(np.array(ansatz_name) == an)[0][0]
        ansatz_chosen = ansatz_lst[an_val]
        
        ic = hyperparam['initial_circuit']
        ic_val = np.where(np.array(initial_circuit_name) == ic)[0][0]
        initial_circuit_chosen = initial_circuit_lst[ic_val]
        
    
    if display_dict is not None:
        display_dict['ansatz_chosen'] = ansatz_name[idx_ansatz]
        display_dict['initial_circuit_chosen'] = initial_circuit_name[idx_initial_circuit]
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return ansatz_chosen, initial_circuit_chosen, display_dict_new

In [11]:
# 5. Get optimizer
#@get_err_msg('')
#@timeit
def _5_get_solver_optimizer(display_dict, problem, hyperparam, qubit_op, qubit_converter, solver_type, ansatz_chosen, initial_circuit_chosen, optimizer, method):
    
    # =============================
    # 5. Define VQE solvers to solve for PES in quantum chemistry
    # =============================
    
    # a. Optimizer set
    # [SPSA(maxiter=100), SLSQP(maxiter=100)] + [partial(minimize, method=i) for i in method_lst]
    # =============================
    method_lst = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr']
    opt = [partial(minimize, method=i) for i in method_lst]

    optimizer_lst = [SPSA(maxiter=100), SLSQP(maxiter=100)] + opt
    optimizer_name = ['SPSA', 'SLSQP'] + ['minimize_{}'.format(i) for i in method_lst]
    
    optimizer_dict = dict(zip(optimizer_name, optimizer_lst))
    optimizer_chosen = optimizer_dict[optimizer]
    
    # Fixed solver and optimizers
    # ['vqe_qnspsa', 'vqe_simulator_noiseless', 'vqe_simulator_noisy_0_0', 'vqe_runtime_noiseless', 'vqe_runtime_noisy_0_0', 'vqe_fake_runtime_noiseless', 
    # 'vqe_fake_runtime_noisy_0_0', 'vqe_ansatz', 'numpy_solver_with_filter', 'numpy_solver', 'vqe_initial_circuit']
    # =============================
    if solver_type == 'vqe_qnspsa':
        ans = initial_circuit_chosen
        obs = qubit_op
        initial_point = np.random.random(ans.num_parameters)

        # loss function
        est = Estimator()

        def loss(x):
            result = est.run([ans], [obs], [x]).result()
            return np.real(result.values[0])

        # fidelity for estimation of the geometric tensor
        sam = Sampler()
        fidelity = QNSPSA.get_fidelity(ans, sam)

        # run QN-SPSA
        solver = QNSPSA(fidelity, maxiter=3000, perturbation = 0.7, learning_rate = 0.01)
        ground_state = solver.optimize(ans.num_parameters, loss, initial_point=initial_point)
        energy = ground_state[1]

    elif solver_type == 'vqe_simulator_noiseless':
        options = {'optimization_level': 3, 'resilience_level': 3}

        ans = initial_circuit_chosen
        opt = optimizer_chosen
        est = Estimator(options = options)

        solver = VQE(ansatz = ans, estimator = est, optimizer = opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    elif solver_type == 'vqe_simulator_noisy_0_0':
        options = {'optimization_level': 0, 'resilience_level': 0}

        ans = initial_circuit_chosen
        opt = optimizer_chosen
        est = Estimator(options = options)

        solver = VQE(ansatz = ans, estimator = est, optimizer = opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    elif solver_type == 'vqe_runtime_noiseless':
        # Other options available: options = {'optimization_level': 0-3, 'resilience_level': 0-3}
        ans = initial_circuit_chosen
        opt = optimizer_chosen

        device = provider.get_backend(_0_get_min_runtime_provider(provider = provider, min_qubits = qubit_op.num_qubits))
        coupling_map = device.configuration().coupling_map
        noise_model = None

        noisy_est = AerEstimator(backend_options={"method": "statevector", "coupling_map": coupling_map, "noise_model": noise_model,},
                                 run_options={"seed": _SEED, "shots": 1024, 'optimization_level': 3, 'resilience_level': 3},
                                 transpile_options={"seed_transpiler": _SEED},
                                )
        solver = VQE(noisy_est, ans, optimizer=opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    elif solver_type == 'vqe_runtime_noisy_0_0':
        # Other options available: options = {'optimization_level': 0-3, 'resilience_level': 0-3}
        ans = initial_circuit_chosen
        opt = optimizer_chosen

        device = provider.get_backend(_0_get_min_runtime_provider(provider = provider, min_qubits = qubit_op.num_qubits))
        coupling_map = device.configuration().coupling_map
        noise_model = NoiseModel.from_backend(device)

        noisy_est = AerEstimator(backend_options={"method": "density_matrix", "coupling_map": coupling_map, "noise_model": noise_model,},
                                 run_options={"seed": _SEED, "shots": 1024, 'optimization_level': 0, 'resilience_level': 0},
                                 transpile_options={"seed_transpiler": _SEED},
                                )
        solver = VQE(noisy_est, ans, optimizer=opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    elif solver_type == 'vqe_fake_runtime_noiseless':
        # Other options available: options = {'optimization_level': 0-3, 'resilience_level': 0-3}
        provider = fake_provider
        ans = initial_circuit_chosen
        opt = optimizer_chosen

        device = provider.get_backend(_0_get_min_runtime_provider(provider = provider, min_qubits = qubit_op.num_qubits))
        coupling_map = device.configuration().coupling_map
        noise_model = None #NoiseModel.from_backend(device)

        noisy_est = AerEstimator(backend_options={"method": "density_matrix", "coupling_map": coupling_map, "noise_model": noise_model,},
                                 run_options={"seed": _SEED, "shots": 1024, 'optimization_level': 3, 'resilience_level': 3},
                                 transpile_options={"seed_transpiler": _SEED},
                                )
        solver = VQE(noisy_est, ans, optimizer=opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    elif solver_type == 'vqe_fake_runtime_noisy_0_0':
        # Other options available: options = {'optimization_level': 0-3, 'resilience_level': 0-3}
        provider = fake_provider
        ans = initial_circuit_chosen
        opt = optimizer_chosen

        device = provider.get_backend(_0_get_min_runtime_provider(provider = provider, min_qubits = qubit_op.num_qubits))
        coupling_map = device.configuration().coupling_map
        noise_model = NoiseModel.from_backend(device)

        noisy_est = AerEstimator(backend_options={"method": "density_matrix", "coupling_map": coupling_map, "noise_model": noise_model,},
                                 run_options={"seed": _SEED, "shots": 1024, 'optimization_level': 0, 'resilience_level': 0},
                                 transpile_options={"seed_transpiler": _SEED},
                                )
        solver = VQE(noisy_est, ans, optimizer=opt)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = ground_state.eigenvalue

    # Customizable solver and optimizers
    # =============================
    # Solvers
    elif solver_type == 'vqe_ansatz':
        est = Estimator()
        ans = ansatz_chosen
        opt = optimizer_chosen

        solver = VQEUCCFactory(est, ans, opt)

    elif solver_type == 'numpy_solver_with_filter':
        solver = NumPyMinimumEigensolverFactory(use_default_filter_criterion=True)

    elif solver_type == 'numpy_solver':
        solver = NumPyMinimumEigensolverFactory()

    elif solver_type == 'vqe_initial_circuit':
        est = Estimator()
        ic = initial_circuit_chosen
        opt = optimizer_chosen

        solver = VQE(est, ic, opt) 

    # Optimizers
    if method == 'gses':
        calc = GroundStateEigensolver(qubit_converter, solver)
        ground_state = calc.solve(problem)

        energy = ground_state.total_energies[0].real

    elif method == 'qpe':
        # Quantum Phase Estimation
        quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))
        evolution = PauliTrotterEvolution('trotter', reps = hyperparam['qpe_num_time_slices'])

        qpe = HamiltonianPhaseEstimation(hyperparam['qpe_n_ancilliae'], quantum_instance=quantum_instance)

        state_preparation = None
        # state_preparation = 
        ground_state = qpe.estimate(qubit_op, state_preparation, evolution=evolution)

        energy = ground_state.most_likely_eigenvalue


    
    # =============================
    
    if display_dict is not None:
        display_dict['solver'] = '{}'.format(solver_type)
        display_dict['optimizer'] = '{}'.format(optimizer)
        display_dict['solution_method'] = '{}'.format(method)
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
        
    return ground_state, energy, display_dict_new


# 2-2. Build construct pipeline

In [12]:
# Create construct problem
#@get_err_msg('')
#@timeit
def get_construct_problem(input_value, hyperparam, input_type, display_report, reduced, basis, mapper_type, solver_type, method, optimizer, seed, randomize=False):
    
    # =============================
    # 0. Display Report Dict
    # =============================
    if display_report == True:
        display_dict = {}
    else:
        display_dict = None
    
    # =============================
    # A. The quantum solver pipeline
    # =============================
    
    # Specify input_type = 'molecule' or 'moleculeinfo'
    problem, display_dict_1 = _1_get_problem_driver(display_dict, input_value, basis = basis, input_type = input_type)
    print('1 OK')
    
    problem, display_dict_2 = _2_get_problem_transform(display_dict_1, problem, input_value, reduced)
    print('2 OK')
    
    qubit_op, qubit_converter, display_dict_3 = _3_get_qubit_operator(display_dict_2, problem, hyperparam, mapper_type)
    print('3 OK')
    
    ansatz_chosen, initial_circuit_chosen, display_dict_4 = _4_get_ansatz(display_dict_3, problem, hyperparam, qubit_op, qubit_converter, seed, randomize)
    print('4 OK')
    
    ground_state, energy, display_dict_5 = _5_get_solver_optimizer(display_dict_4, problem, hyperparam, qubit_op, qubit_converter, 
                                                                   solver_type, ansatz_chosen, initial_circuit_chosen, optimizer, method)
    print('5 OK')
    
    if display_report == True:
        display_df = pd.DataFrame.from_dict([display_dict_5])
    else:
        display_df = None
    
    return ground_state, energy, display_df


# ======================================

# 3-1. Calculate molecule energy by BOPES calculations with regressed distance

In [13]:
def get_a_molecule_perturbation_list(moleculeinfo, hyperparam, perturbation_steps):
    
    from copy import deepcopy
    
    EPSILON = 1e-3
    
    atom_pair = hyperparam['atom_pair']
    pair_0, pair_1 = atom_pair
    
    x0, y0, z0 = moleculeinfo.coords[pair_0 - 1]
    x1, y1, z1 = moleculeinfo.coords[pair_1 - 1]
    
    # Get a straight line connecting the 2 interacting atom pair, y = mx + p
    m = 0
    p = y0
    if abs(x1 - x0) > EPSILON:
        m = (y1 - y0)/(x1 - x0)
        p = y0 - m*x0
    
    # This perturbation assumes lying on the same plane
    size = len(perturbation_steps)
    energy_lst = np.empty(size)
    
    perturbation_lst = []
    for k in range(size):
        # print("Step: ", k)
        
        if (abs(x0) < EPSILON and abs(y0) < EPSILON):
            z0_new = z0 + perturbation_steps[k]
            
            coords_new = []
            for l in range(len(moleculeinfo.coords)):
                if l == atom_pair[0]:
                    coords_new.append((0.0, 0.0, z0_new))
                else:
                    coords_new.append(moleculeinfo.coords[l])
        
        elif (abs(z0) < EPSILON and abs(z1) < EPSILON):
            x0_new = x0 + perturbation_steps[k]
            y0_new = m*x0_new + p
            
            coords_new = []
            for l in range(len(moleculeinfo.coords)):
                if l == atom_pair[0]:
                    coords_new.append((x0_new, y0_new, 0.0))
                else:
                    coords_new.append(moleculeinfo.coords[l])
                    
        else:
            print("bopes - Error: unsupported molecule geometry, atom pairs must be in the same line or in the same plane")
            return perturbation_steps, 0
    
        moleculeinfo_new = deepcopy(moleculeinfo)
        moleculeinfo_new.coords = coords_new
        
        perturbation_lst.append(moleculeinfo_new)
        
    return perturbation_lst

In [15]:
# Get BOPES energy curve

def get_a_molecule_bopes(input_value, hyperparam, input_type = 'molecule',
                       reduced = 'ActiveSpace', basis = 'sto3g', mapper_type = 'JordanWignerMapper', solver_type = 'numpy_solver', method = 'gses', 
                       perturbation_steps = np.linspace(0.5, 3, 250), optimizer = 'SLSQP', display_report = False, seed = _SEED, randomize = False):
    
    # Obtain molecule coordinates
    # ===========================
    EPSILON = 1e-3
    
    if input_type == 'molecule':
        moleculeinfo = MoleculeInfo(symbols = input_value['symbols'], coords = input_value['coords'], masses = input_value['masses'], 
                                    charge = input_value['charge'], multiplicity = input_value['multiplicity'])
    
    elif input_type == 'moleculeinfo':
        moleculeinfo = input_value
    
    perturbation_lst = get_a_molecule_perturbation_list(moleculeinfo, hyperparam, perturbation_steps)
    
    energy_lst = []
    for k in tqdm(range(len(perturbation_lst)), position = 0, leave = True):
        
        tmp_moleculeinfo = perturbation_lst[k]
        
        # Get ground state energy from construct problem
        # ===========================
        _, tmp_energy, _ = get_construct_problem(input_value, hyperparam, input_type, display_report, reduced, basis, mapper_type, solver_type, method, optimizer, seed, randomize)
        energy_lst.append(tmp_energy)
        
    if display_report:
        from matplotlib import pyplot as plt
        %matplotlib inline
        plt.rcParams["figure.figsize"] = [10, 10]
        plt.rcParams["figure.autolayout"] = True
        
        df = pd.DataFrame(list(zip(energy_lst, perturbation_steps)), columns = ['step', 'energy'])
        fig, ax = plt.subplots(facecolor='lightslategray')
        ax.set_clip_on(False)
        df.plot(kind='scatter', x='energy', y='step', ax=ax, color='black', linewidth=0, label = 'VQE Energy (HF)')
        
        e_min = min(energy_lst)
        p_min = perturbation_steps[energy_lst == e_min]
        print('Molecule energy:{} with atoms separated at {} apart'.format(e_min, p_min))
        
    return perturbation_steps, energy_lst


# 3-2. Calculate molecule energy by BOPES calculations with Variable MOF atomic distance relative to gas molecule

# 3-3. Unittest for molecule energy from different parameter combinations

In [18]:
# ===
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           # 'atom_pair': (1, 2), # not available after migration
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           }
}

hyperparameters = {
    'h2': {'reps': 2,
           'two_qubit_reduction': True,
           'z2symmetry_reduction': 'auto',
           'perturbation_steps': np.linspace(0.5, 4.0, 250),
           'qpe_num_time_slices': 1,
           'qpe_n_ancilliae': 3,
           'atom_pair': (1, 2), # index start from 1
           'optimizer': 'SLSQP',
           'ansatz': 'UCCSD',
           'initial_circuit': 'PauliTwoDesign',
           'qnspsa_pertubation': 0.7,
           'qnspsa_learning_rate': 0.01,
          },

}

solution_master = {
    'classical': ['numpy_solver', 'numpy_solver_with_filter'],
    'quantum': ['vqe_qnspsa', 'vqe_runtime', 'vqe_runtime_with_noise', 'vqe_ansatz', 'vqe_ansatz', 'vqe_initial_circuit']
}

# ===
molecule = gas_molecules['h2']
hyperparam = hyperparameters['h2']
# display_report = False
display_dict = {}
basis = 'sto3g'


In [19]:
lst_mapper_type = ['JordanWignerMapper']
# lst_reduced = ['FreezeCore', 'ActiveSpace', '']
lst_reduced = ['FreezeCore']

# method_lst = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr']
# lst_optimizer = ['SPSA', 'SLSQP'] + ['minimize_{}'.format(i) for i in method_lst]
# lst_optimizer = ['SPSA', 'SLSQP']
lst_optimizer = ['SPSA']

# lst_solver_type = ['vqe_qnspsa', 'vqe_runtime_noiseless', 'vqe_runtime_noisy_0', 'numpy_solver_with_filter', 'numpy_solver', 'vqe_ansatz', 'vqe_initial_circuit']
lst_solver_type = ['vqe_fake_runtime_noiseless']

#lst_method = ['gses', 'qpe']
lst_method = ['gses']

#initial_circuit_name = ['TwoLocal', 'EfficientSU2', 'PauliTwoDesign', 'RealAmplitudes', 'ExcitationPreserving']
hyperparam['initial_circuit'] = 'RealAmplitudes'
# hyperparam['fake_provider'] = fake_provider

for a in lst_mapper_type:
    for b in lst_reduced:
        for c in lst_optimizer:
            for d in lst_solver_type:
                for e in lst_method:
                    tmp_ground_state, tmp_energy, _ = get_construct_problem(input_value = gas_molecules['h2'], hyperparam = hyperparameters['h2'], input_type = 'molecule', 
                                                             display_report = False, reduced = b, basis = basis, mapper_type = a, 
                                                             solver_type = d, method = e, optimizer = c, seed = _SEED, randomize = False)
                    print('{}|{}|{}|{}|{}: {}'.format(a, b, c, d, e, tmp_ground_state))
                    

1 OK
2 OK
3 OK
4 OK
Noisy Aer provider with min_qubit chosen: 2, fake_openpulse_2q
5 OK
JordanWignerMapper|FreezeCore|SPSA|vqe_fake_runtime_noiseless|gses: === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.838130413569
  - computed part:      -1.838130413569
  - FreezeCoreTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.713753993688
> Total ground state energy (Hartree): -1.124376419881
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.40104295]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  None]
    - computed part:      [0.0  0.0  None]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  None]  Total: None
                 (debye): [0.0  0.0  None]  Total: None
 


# 4. Deployment

In [ ]:
# ===
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           # 'atom_pair': (1, 2), # not available after migration
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           }
}

hyperparameters = {
    'h2': {'reps': 2,
           'two_qubit_reduction': True,
           'z2symmetry_reduction': 'auto',
           'perturbation_steps': np.linspace(0.5, 4.0, 250),
           'qpe_num_time_slices': 1,
           'qpe_n_ancilliae': 3,
           'atom_pair': (1, 2), # index start from 1
          },

}

solution_master = {
    'classical': ['numpy_solver', 'numpy_solver_with_filter'],
    'quantum': ['vqe_qnspsa', 'vqe_runtime', 'vqe_runtime_with_noise', 'vqe_ansatz', 'vqe_ansatz', 'vqe_initial_circuit']
}

# ===
input_value = gas_molecules['h2']
hyperparam = hyperparameters['h2']
input_type = 'molecule'
display_report = True
display_dict = {}
reduced = 'FreezeCore'
basis = 'sto3g'
mapper_type = 'JordanWignerMapper'
solver_type = 'numpy_solver'
method = 'gses'
seed = _SEED

In [ ]:
_, energy, display_df = get_construct_problem(input_value, hyperparam, input_type, display_report, reduced, basis, mapper_type, solver_type, method, optimizer, seed)
display_df

In [ ]:
perturb_steps, energy_lst = get_molecule_bopes(input_value = gas_molecules['h2'], hyperparam = hyperparameters['h2'], input_type = 'molecule',
                                               reduced = 'FreezeCore', basis = 'sto3g', mapper_type = 'JordanWignerMapper', solver_type = 'numpy_solver', method = 'gses', 
                                               perturbation_steps = np.linspace(0.5, 3.0, 250), display_report = True, seed = _SEED)

In [ ]:
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           },
    'n2': {'symbols': ["N", "N"],
           'coords': [(0.0000, 0.0000, 0.5488), 
                      (0.0000, 0.0000, -0.5488)],
           'multiplicity': 4,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [7, 7],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           
           
           },
    'co2': {'symbols': ["C", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 0.0000, 1.1621),
                       (0.0000, 0.0000, -1.1621)],
            'multiplicity': 2,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'h2o': {'symbols': ["O", "H", "H"],
            'coords': [(0.0000, 0.0000, 0.1173),
                       (0.0000, 0.7572, -0.4692),
                       (0.0000, -0.7572, -0.4692)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'so2': {'symbols': ["S", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 1.2371, 0.7215),
                       (0.0000, -1.2371, 0.7215)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'no2': {'symbols': ["N", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 1.0989, 0.4653),
                       (0.0000, -1.0989, 0.4653)],
            'multiplicity': 2,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'n2o': {'symbols': ["N", "N", "O"],
            'coords': [(0.0000, 0.0000, -1.1998),
                       (0.0000, 0.7572, -0.0716),
                       (0.0000, -0.7572, 1.126)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'ch4': {'symbols': ["C", "H", "H", "H", "H"],
            'coords': [(0.0000, 0.0000, 0.0000), 
                       (0.6276, 0.6276, 0.6276),
                       (0.6276, -0.6276, -0.6276),
                       (-0.6276, 0.6276, -0.6276),
                       (0.6276, -0.6276, 0.6276)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 1, 1, 1, 1],
            'atom_pair': (1, 4)
           },
    

}

# Not updated
# Spin multiplicity check SO2, N2O, CH4
